In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "virny-flow-experiments":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/Research/NYU/VirnyFlow_Project/Code/virny-flow-experiments


# Case Studies Visualizations

In [4]:
import pandas as pd
from duckdb import query as sqldf
from virny_flow.core.custom_classes.core_db_client import CoreDBClient
from virny_flow.configs.constants import EXP_CONFIG_HISTORY_TABLE
from source.visualizations.use_case_queries import get_best_lps_per_exp_config
from source.visualizations.scalability_viz import create_performance_plot_v3

## Prepare data for visualizations

In [5]:
DATASET_NAME = 'heart'
DISPARITY_METRIC = 'Equalized_Odds_TNR'
GROUP = "gender"

# DATASET_NAME = 'folk_pubcov'
# DISPARITY_METRIC1 = 'Selection_Rate_Difference'
# GROUP1 = "SEX"
# DISPARITY_METRIC2 = 'Selection_Rate_Difference'
# GROUP2 = "RAC1P"

In [6]:
SECRETS_PATH = os.path.join(os.getcwd(), "scripts", "configs", "secrets.env")
EXP_NAME = 'scalability_exp'
VIRNY_FLOW = 'virny_flow'
ALPINE = 'alpine_meadow'
AUTOSKLEARN = 'autosklearn'
EXP_CONFIG_NAMES = {
    VIRNY_FLOW: {
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_1': 1,
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_2': 2,
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_4': 4,
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_8': 8,
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_16': 16,
        f'sensitivity_exp_{DATASET_NAME}_w32_vf_pp_cand_32': 32,
    },
}

db_client = CoreDBClient(SECRETS_PATH)
db_client.connect()

In [7]:
def get_virny_flow_metrics(db_client):
    exp_config_names = list(EXP_CONFIG_NAMES['virny_flow'].keys())
    best_lp_metrics_per_exp_config_df = get_best_lps_per_exp_config(secrets_path=SECRETS_PATH,
                                                                    exp_config_names=exp_config_names)
    best_lp_metrics_per_exp_config_df['num_pp_candidates'] = best_lp_metrics_per_exp_config_df['exp_config_name'].map(EXP_CONFIG_NAMES['virny_flow'])

    virny_flow_all_runtime_df = pd.DataFrame()
    for exp_config_name in exp_config_names:
        virny_flow_runtime_df = db_client.read_metric_df_from_db(collection_name=EXP_CONFIG_HISTORY_TABLE,
                                                                 query={'exp_config_name': exp_config_name,
                                                                        'deletion_flag': False})
        virny_flow_all_runtime_df = pd.concat([virny_flow_all_runtime_df, virny_flow_runtime_df])
    
    new_column_names = []
    for col in virny_flow_all_runtime_df.columns:
        new_col_name = '_'.join([c.lower() for c in col.split('_')])
        new_column_names.append(new_col_name)
    virny_flow_all_runtime_df.columns = new_column_names

    virny_flow_metrics_df = sqldf("""
        SELECT DISTINCT t1.*, t2.exp_config_execution_time
        FROM best_lp_metrics_per_exp_config_df AS t1
        JOIN virny_flow_all_runtime_df AS t2
          ON t1.exp_config_name = t2.exp_config_name
         AND t1.run_num = t2.run_num
    """).to_df()
    
    return virny_flow_metrics_df

In [8]:
virny_flow_metrics_df = get_virny_flow_metrics(db_client)

Extracting metrics for sensitivity_exp_heart_w32_vf_pp_cand_1...
best_pps_per_lp_and_run_num_df.shape: (162, 15)
best_lp_per_run_all.shape: (81, 15)
Extracted metrics for sensitivity_exp_heart_w32_vf_pp_cand_1

Extracting metrics for sensitivity_exp_heart_w32_vf_pp_cand_2...
best_pps_per_lp_and_run_num_df.shape: (189, 15)
best_lp_per_run_all.shape: (90, 15)
Extracted metrics for sensitivity_exp_heart_w32_vf_pp_cand_2

Extracting metrics for sensitivity_exp_heart_w32_vf_pp_cand_4...
best_pps_per_lp_and_run_num_df.shape: (180, 15)
best_lp_per_run_all.shape: (81, 15)
Extracted metrics for sensitivity_exp_heart_w32_vf_pp_cand_4

Extracting metrics for sensitivity_exp_heart_w32_vf_pp_cand_8...
best_pps_per_lp_and_run_num_df.shape: (144, 15)
best_lp_per_run_all.shape: (72, 15)
Extracted metrics for sensitivity_exp_heart_w32_vf_pp_cand_8

Extracting metrics for sensitivity_exp_heart_w32_vf_pp_cand_16...
best_pps_per_lp_and_run_num_df.shape: (144, 15)
best_lp_per_run_all.shape: (72, 15)
Extrac

## Display Results

In [9]:
from virny_flow.visualizations.use_case_queries import get_models_disparity_metric_df


def display_table_with_results(system_metrics_df, system_name: str, disparity_metric_name: str, group_name: str):
    if system_name == VIRNY_FLOW:
        system_metrics_df['system_name'] = system_name
        common_cols = ['system_name', 'dataset_name', 'num_pp_candidates', 'run_num', 'exp_config_execution_time']
    else:
        common_cols = ['system_name', 'dataset_name', 'num_pp_candidates', 'run_num', 'optimization_time']

    f1_metrics_df = system_metrics_df[system_metrics_df['metric'] == 'F1']
    f1_metrics_df['F1'] = f1_metrics_df['overall']
    f1_metrics_df = f1_metrics_df[common_cols + ['F1']]

    disparity_metric_df = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name, group_name)
    disparity_metric_df[disparity_metric_name] = disparity_metric_df['disparity_metric_value']
    disparity_metric_df = disparity_metric_df[common_cols + [disparity_metric_name]]

    final_metrics_df = sqldf(f"""
        SELECT t1.*, t2.{disparity_metric_name}
        FROM f1_metrics_df AS t1
        JOIN disparity_metric_df AS t2
          ON t1.run_num = t2.run_num
         AND t1.num_pp_candidates = t2.num_pp_candidates
    """).to_df()
    final_metrics_df["score"] = final_metrics_df["F1"] * 0.5 + (1 - abs(final_metrics_df[disparity_metric_name])) * 0.5

    if system_name == VIRNY_FLOW:
        final_metrics_df = final_metrics_df[~final_metrics_df['exp_config_execution_time'].isna()]
        final_metrics_df = final_metrics_df.rename(columns={'exp_config_execution_time': 'optimization_time'})

    return final_metrics_df


def display_table_with_results_for_folk_pubcov(system_metrics_df, system_name: str,
                                               disparity_metric_name1: str, group_name1: str,
                                               disparity_metric_name2: str, group_name2: str):
    if system_name == VIRNY_FLOW:
        system_metrics_df['system_name'] = system_name
        common_cols = ['system_name', 'dataset_name', 'num_pp_candidates', 'run_num', 'exp_config_execution_time']
    else:
        common_cols = ['system_name', 'dataset_name', 'num_pp_candidates', 'run_num', 'optimization_time']

    f1_metrics_df = system_metrics_df[system_metrics_df['metric'] == 'F1']
    f1_metrics_df['F1'] = f1_metrics_df['overall']
    f1_metrics_df = f1_metrics_df[common_cols + ['F1']]

    disparity_metric_df1 = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name1, group_name1)
    disparity_metric_df1[disparity_metric_name1] = disparity_metric_df1['disparity_metric_value']
    disparity_metric_df1 = disparity_metric_df1[common_cols + [disparity_metric_name1]]

    disparity_metric_df2 = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name2, group_name2)
    disparity_metric_df2[disparity_metric_name2] = disparity_metric_df2['disparity_metric_value']
    disparity_metric_df2 = disparity_metric_df2[common_cols + [disparity_metric_name2]]

    final_metrics_df = sqldf(f"""
            SELECT t1.*, t2.{disparity_metric_name1} AS {disparity_metric_name1}_{group_name1}, t3.{disparity_metric_name2} AS {disparity_metric_name2}_{group_name2}
            FROM f1_metrics_df AS t1
            JOIN disparity_metric_df1 AS t2
              ON t1.run_num = t2.run_num
             AND t1.dataset_name = t2.dataset_name
             AND t1.num_pp_candidates = t2.num_pp_candidates
            JOIN disparity_metric_df2 AS t3
              ON t1.run_num = t3.run_num
             AND t1.dataset_name = t3.dataset_name
             AND t1.num_pp_candidates = t3.num_pp_candidates
        """).to_df()
    
    final_metrics_df["score"] = (final_metrics_df["F1"] * 0.33 + 
                                 (1 - abs(final_metrics_df[f"{disparity_metric_name1}_{group_name1}"])) * 0.33 + 
                                 (1 - abs(final_metrics_df[f"{disparity_metric_name2}_{group_name2}"])) * 0.33)

    if system_name == VIRNY_FLOW:
        final_metrics_df = final_metrics_df[~final_metrics_df['exp_config_execution_time'].isna()]        
        final_metrics_df = final_metrics_df.rename(columns={'exp_config_execution_time': 'optimization_time'})

    return final_metrics_df


def create_latex_table(df):
    # Compute mean and std for each system
    summary = df.groupby('num_pp_candidates').agg(['mean', 'std']).round(4)

    # Combine mean and std into "mean ± std" format
    def format_metric(mean, std):
        if pd.isna(std):
            return f"{mean:.4f} ± n/a"
        return f"{mean:.4f} \scriptsize{{$\pm${std:.4f}}}"

    def format_runtime(mean, std):
        if pd.isna(std):
            return f"{mean:.0f} ± n/a"
        return f"{mean:.0f} \scriptsize{{$\pm${std:.0f}}}"

    # Create formatted DataFrame
    latex_df = pd.DataFrame({
        '# of PP Candidates': summary.index,
        'Score': [format_metric(m, s) for m, s in zip(summary['score']['mean'], summary['score']['std'])],
        'Runtime': [format_runtime(m, s) for m, s in zip(summary['optimization_time']['mean'], summary['optimization_time']['std'])],
    })

    # Reorder rows: virny_flow first
    latex_df = latex_df.set_index('# of PP Candidates').loc[[1, 2, 4, 8, 16, 32]].reset_index()

    # Generate LaTeX table
    latex_table = latex_df.to_latex(index=False,
                                    caption='Sensitivity to the Number of Physical Pipeline Candidates',
                                    label='tab:sensitivity_pp',
                                    column_format='lcc',
                                    escape=False)

    print(latex_table)

In [10]:
if DATASET_NAME == 'folk_pubcov':
    virny_flow_final_metrics_df = display_table_with_results_for_folk_pubcov(virny_flow_metrics_df, 'virny_flow',
                                                                             DISPARITY_METRIC1, GROUP1,
                                                                             DISPARITY_METRIC2, GROUP2)
else:
    virny_flow_final_metrics_df = display_table_with_results(virny_flow_metrics_df, 'virny_flow', DISPARITY_METRIC, GROUP)

In [11]:
virny_flow_final_metrics_df[virny_flow_final_metrics_df['num_pp_candidates'] == 8]

,system_name,dataset_name,num_pp_candidates,run_num,optimization_time,F1,Equalized_Odds_TNR,score
2,virny_flow,heart,8,2,590.595077,0.728010,-0.001241,0.863385
3,virny_flow,heart,8,3,1159.774059,0.723501,0.001325,0.861088
5,virny_flow,heart,8,8,827.847077,0.720805,0.000803,0.860001
50,virny_flow,heart,8,1,565.025150,0.728869,-0.004557,0.862156


In [12]:
create_latex_table(virny_flow_final_metrics_df)

\begin{table}
\centering
\caption{Sensitivity to the Number of Physical Pipeline Candidates}
\label{tab:sensitivity_pp}
\begin{tabular}{lcc}
\toprule
 # of PP Candidates &                           Score &                    Runtime \\
\midrule
                  1 & 0.8631 \scriptsize{$\pm$0.0020} & 1049 \scriptsize{$\pm$461} \\
                  2 & 0.8630 \scriptsize{$\pm$0.0021} &  917 \scriptsize{$\pm$180} \\
                  4 & 0.8628 \scriptsize{$\pm$0.0033} &  781 \scriptsize{$\pm$104} \\
                  8 & 0.8617 \scriptsize{$\pm$0.0014} &  786 \scriptsize{$\pm$276} \\
                 16 & 0.8608 \scriptsize{$\pm$0.0061} &  724 \scriptsize{$\pm$175} \\
                 32 & 0.8595 \scriptsize{$\pm$0.0091} &  585 \scriptsize{$\pm$165} \\
\bottomrule
\end{tabular}
\end{table}



In [13]:
create_performance_plot_v3(virny_flow_final_metrics_df, metric_name="F1")

alt.Chart(...)

In [14]:
create_performance_plot_v3(virny_flow_final_metrics_df, metric_name=DISPARITY_METRIC1 + '_' + GROUP1)

NameError: name 'DISPARITY_METRIC1' is not defined

In [15]:
create_performance_plot_v3(virny_flow_final_metrics_df, metric_name="score")

alt.Chart(...)